In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time
from tqdm import tqdm  # 진행 바 표시용
import sys
import os

# models 디렉토리 추가
sys.path.append('./models')
from resnet import resnet18  # 커스텀 ResNet 모델 임포트

# CUDA 메모리 관리 최적화 설정
torch.backends.cudnn.benchmark = False  # 벤치마크 끄기
torch.backends.cudnn.deterministic = True  # 결정적 알고리즘 사용
torch.cuda.empty_cache()  # CUDA 캐시 비우기

# 디바이스 설정 - CPU 옵션 추가
force_cpu = False  # True로 설정하면 강제로 CPU 사용
device = torch.device("cpu") if force_cpu else torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU 수: {torch.cuda.device_count()}")
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
    print(f"가용 CUDA 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# 테스트 데이터셋 설정 (CIFAR-100)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(
    testset, 
    batch_size=256,  # 배치 크기 조정
    shuffle=False, 
    pin_memory=(device.type == 'cuda'),  # GPU 사용시에만 pin_memory 활성화
    num_workers=32  # 시스템에 맞게 조정
)

# 배치 수 계산하여 출력
total_batches = len(testloader)
print(f"총 테스트 배치 수: {total_batches}, 배치 크기: {testloader.batch_size}")
print(f"총 테스트 샘플 수: {len(testset)}")

# 모델 평가 함수
def evaluate_model(model_path, model_name="ResNet18"):
    try:
        start_time = time.time()
        print(f"\n[{time.strftime('%H:%M:%S')}] {model_name} 평가 시작...")
        
        # 모델 생성 - 커스텀 ResNet18 사용
        print(f"[{time.strftime('%H:%M:%S')}] 모델 생성 중...")
        model = resnet18()  # 이미 CIFAR-100용으로 설정되어 있음 (num_classes=100)
        
        # 저장된 가중치를 불러옴
        print(f"[{time.strftime('%H:%M:%S')}] 모델 가중치 로딩 중...")
        if device.type == 'cuda':
            state_dict = torch.load(model_path, map_location=device)
        else:
            state_dict = torch.load(model_path, map_location='cpu')
        
        # 'module.' 접두사 처리
        if all(k.startswith('module.') for k in state_dict.keys()):
            # GPU 사용 가능하고 여러 개일 경우 DataParallel 사용
            if device.type == 'cuda' and torch.cuda.device_count() > 1 and not force_cpu:
                model = nn.DataParallel(model)
            else:
                # CPU 모드이거나 GPU가 하나일 경우 접두사 제거
                state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
        
        # 가중치 로드
        model.load_state_dict(state_dict)
        print(f"[{time.strftime('%H:%M:%S')}] 모델 가중치 로딩 완료")
        
        # 디바이스로 모델 이동
        model = model.to(device)
        model.eval()
        
        correct_top1 = 0
        correct_top5 = 0
        total = 0
        
        cpu_fallback_count = 0  # CPU로 전환한 배치 수
        error_batch_count = 0   # 오류가 발생한 배치 수
        
        # 평가 과정에서 에러 핸들링 + 진행 상태 표시
        with torch.no_grad():
            # tqdm을 사용하여 진행 바 표시
            pbar = tqdm(testloader, desc=f"{model_name} 평가 중", unit="batch")
            for batch_idx, (inputs, labels) in enumerate(pbar):
                try:
                    # 5배치마다 진행 상황 출력
                    if batch_idx % 5 == 0:
                        print(f"[{time.strftime('%H:%M:%S')}] 배치 {batch_idx}/{total_batches} 처리 중...")
                    
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    
                    # Top-1 정확도
                    _, predicted = outputs.max(1)
                    batch_correct = predicted.eq(labels).sum().item()
                    total += labels.size(0)
                    correct_top1 += batch_correct
                    
                    # Top-5 정확도
                    _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
                    batch_correct_top5 = top5_idx.eq(labels.view(-1, 1).expand_as(top5_idx)).sum().item()
                    correct_top5 += batch_correct_top5
                    
                    # 현재 배치 정확도를 진행 바에 업데이트
                    batch_acc = 100. * batch_correct / labels.size(0)
                    pbar.set_postfix({"배치 정확도": f"{batch_acc:.2f}%", 
                                     "전체 정확도": f"{100. * correct_top1 / total:.2f}%"})
                    
                except RuntimeError as e:
                    print(f"\n[{time.strftime('%H:%M:%S')}] 배치 {batch_idx}/{total_batches} 처리 중 오류 발생: {e}")
                    error_batch_count += 1
                    
                    # CPU로 전환하여 재시도
                    if device.type == 'cuda':
                        print(f"[{time.strftime('%H:%M:%S')}] CPU로 전환하여 계속합니다...")
                        cpu_fallback_count += 1
                        inputs, labels = inputs.cpu(), labels.cpu()
                        model = model.cpu()
                        outputs = model(inputs)
                        
                        # Top-1 정확도
                        _, predicted = outputs.max(1)
                        batch_correct = predicted.eq(labels).sum().item()
                        total += labels.size(0)
                        correct_top1 += batch_correct
                        
                        # Top-5 정확도
                        _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
                        batch_correct_top5 = top5_idx.eq(labels.view(-1, 1).expand_as(top5_idx)).sum().item()
                        correct_top5 += batch_correct_top5
                        
                        # 현재 배치 정확도를 진행 바에 업데이트
                        batch_acc = 100. * batch_correct / labels.size(0)
                        pbar.set_postfix({"배치 정확도": f"{batch_acc:.2f}%", 
                                          "전체 정확도": f"{100. * correct_top1 / total:.2f}%",
                                          "CPU 전환": cpu_fallback_count})
                        
                        # 다시 GPU로 전환
                        model = model.to(device)
        
        accuracy_top1 = 100.0 * correct_top1 / total
        accuracy_top5 = 100.0 * correct_top5 / total
        
        evaluation_time = time.time() - start_time
        
        print(f'\n[{time.strftime("%H:%M:%S")}] {model_name} 평가 완료:')
        print(f'- Top-1 정확도: {accuracy_top1:.2f}%')
        print(f'- Top-5 정확도: {accuracy_top5:.2f}%')
        print(f'- 평가 소요 시간: {evaluation_time:.2f}초')
        print(f'- 오류 발생 배치 수: {error_batch_count}/{total_batches}')
        print(f'- CPU 전환 배치 수: {cpu_fallback_count}/{total_batches}')
        
        return accuracy_top1, accuracy_top5
    
    except Exception as e:
        print(f"\n[{time.strftime('%H:%M:%S')}] 모델 평가 중 예외 발생: {e}")
        return 0.0, 0.0

# 모델 가중치 분석 함수
def analyze_model_weights(model_path, model_name="ResNet18"):
    print(f"\n[{time.strftime('%H:%M:%S')}] {model_name} 가중치 분석 중...")
    
    # 파일 크기 확인
    import os
    file_size_mb = os.path.getsize(model_path) / (1024 * 1024)
    print(f"{model_name} 파일 크기: {file_size_mb:.2f} MB")
    
    # 가중치 통계 분석
    state_dict = torch.load(model_path, map_location='cpu')
    
    # 가중치 개수, 평균, 표준편차 계산
    total_params = 0
    total_weight_sum = 0
    total_abs_weight_sum = 0
    weight_std_sum = 0
    weight_count = 0
    
    for name, param in state_dict.items():
        if 'weight' in name:
            weight_count += 1
            param_count = param.numel()
            total_params += param_count
            
            # 가중치 통계
            param_mean = param.mean().item()
            param_abs_mean = param.abs().mean().item()
            param_std = param.std().item()
            
            total_weight_sum += param_mean * param_count
            total_abs_weight_sum += param_abs_mean * param_count
            weight_std_sum += param_std
    
    # 평균값 계산
    avg_weight = total_weight_sum / total_params
    avg_abs_weight = total_abs_weight_sum / total_params
    avg_std = weight_std_sum / weight_count
    
    print(f"{model_name} 총 파라미터 수: {total_params:,}")
    print(f"{model_name} 가중치 평균: {avg_weight:.6f}")
    print(f"{model_name} 가중치 절대값 평균: {avg_abs_weight:.6f}")
    print(f"{model_name} 가중치 표준편차 평균: {avg_std:.6f}")
    
    # 학습된 모델 여부 예측
    is_trained = avg_std > 0.01  # 일반적으로 학습된 모델은 표준편차가 더 큼
    print(f"{model_name}는 {'학습된 모델로 추정됩니다.' if is_trained else '학습되지 않은 모델로 추정됩니다.'}")
    
    return {
        'file_size_mb': file_size_mb,
        'total_params': total_params,
        'avg_weight': avg_weight,
        'avg_abs_weight': avg_abs_weight,
        'avg_std': avg_std,
        'is_trained': is_trained
    }

if __name__ == "__main__":
    model_path = 'best_model_resnet18.pth'
    
    # 모델 파일 존재 확인
    if not os.path.exists(model_path):
        print(f"오류: {model_path} 파일을 찾을 수 없습니다.")
        sys.exit(1)
    
    # models 디렉토리 존재 확인 
    if not os.path.exists('./models'):
        print("오류: models 디렉토리를 찾을 수 없습니다.")
        sys.exit(1)
    
    # models/resnet.py 파일 존재 확인
    if not os.path.exists('./models/resnet.py'):
        print("오류: models/resnet.py 파일을 찾을 수 없습니다.")
        sys.exit(1)
    
    # 1. 먼저 모델 가중치 분석
    analysis_results = analyze_model_weights(model_path, "ResNet18")
    
    # 2. 모델 성능 평가
    if analysis_results['is_trained']:
        print("\n------- ResNet18 모델 평가 시작 -------")
        top1_acc, top5_acc = evaluate_model(model_path, "ResNet18")
        
        print("\n------- 최종 결과 -------")
        print(f"ResNet18 모델의 CIFAR-100 테스트셋 성능:")
        print(f"- Top-1 정확도: {top1_acc:.2f}%")
        print(f"- Top-5 정확도: {top5_acc:.2f}%")
    else:
        print("\n------- 주의 -------")
        print("모델이 제대로 학습되지 않은 것으로 추정됩니다.")
        print("평가를 계속하시겠습니까? (y/n)")
        choice = input()
        if choice.lower() == 'y':
            top1_acc, top5_acc = evaluate_model(model_path, "ResNet18")
            
            print("\n------- 최종 결과 -------")
            print(f"ResNet18 모델의 CIFAR-100 테스트셋 성능:")
            print(f"- Top-1 정확도: {top1_acc:.2f}%")
            print(f"- Top-5 정확도: {top5_acc:.2f}%")

Using device: cuda
GPU 수: 2
GPU 이름: NVIDIA RTX A5000
가용 CUDA 메모리: 25.43 GB
Files already downloaded and verified
총 테스트 배치 수: 40, 배치 크기: 256
총 테스트 샘플 수: 10000

[12:14:50] ResNet18 가중치 분석 중...
ResNet18 파일 크기: 42.89 MB
ResNet18 총 파라미터 수: 11,215,232
ResNet18 가중치 평균: -0.000783
ResNet18 가중치 절대값 평균: 0.007838
ResNet18 가중치 표준편차 평균: 0.036871
ResNet18는 학습된 모델로 추정됩니다.

------- ResNet18 모델 평가 시작 -------

[12:14:50] ResNet18 평가 시작...
[12:14:50] 모델 생성 중...
[12:14:50] 모델 가중치 로딩 중...
[12:14:50] 모델 가중치 로딩 완료


ResNet18 평가 중:   2%|▉                                  | 1/40 [00:01<01:04,  1.65s/batch, 배치 정확도=82.42%, 전체 정확도=82.42%] ?batch/s]

[12:14:52] 배치 0/40 처리 중...


ResNet18 평가 중:  15%|█████▎                             | 6/40 [00:02<00:07,  4.58batch/s, 배치 정확도=78.91%, 전체 정확도=79.91%]

[12:14:52] 배치 5/40 처리 중...


...

[12:14:55] 배치 30/40 처리 중...


ResNet18 평가 중:  90%|██████████████████████████████▌   | 36/40 [00:05<00:00, 10.98batch/s, 배치 정확도=82.42%, 전체 정확도=80.23%]

[12:14:55] 배치 35/40 처리 중...


ResNet18 평가 중: 100%|██████████████████████████████████| 40/40 [00:05<00:00,  7.21batch/s, 배치 정확도=68.75%, 전체 정확도=80.32%]


[12:14:56] ResNet18 평가 완료:
- Top-1 정확도: 80.32%
- Top-5 정확도: 95.50%
- 평가 소요 시간: 5.82초
- 오류 발생 배치 수: 0/40
- CPU 전환 배치 수: 0/40

------- 최종 결과 -------
ResNet18 모델의 CIFAR-100 테스트셋 성능:
- Top-1 정확도: 80.32%
- Top-5 정확도: 95.50%
